In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Naive Bayes


Naive Bayes is a classification algorithm that, as the name implies, uses Bayesian statistics naively.  The goal is, given a set of observed features $X_1, \ldots, X_p$, to find the label $Y$ with the maximum conditional probability.  This will be the prediction of the model.

$$ \begin{align}
p(Y \mid X_1, \ldots, X_p) & = \frac{ p(Y) p( X_1, \ldots, X_p \mid Y) }{ p( X_1, \ldots, X_p) } \\
& \propto p(Y) p( X_1, \ldots, X_p \mid Y) \\
& = p(Y) p(X_1 \mid Y) p(X_2 \mid Y, X_1) \cdots p(X_p \mid Y, X_1, \ldots, X_{p-1}) \\
& \approx p(Y) \prod_{i=1}^p p(X_i \mid Y)
\end{align}$$

In line 2, we drop the denominator, as it is independent of $Y$.  The final step is the naive assumption that all of the $X_i$ are independent; that is that $P(X_i \mid X_j) \approx P(X_i)$.


## Predictive modeling using Naive Bayes


Naive Bayes is most often applied in the case where $X$ and $Y$ are discrete (i.e. categorical data).  Taking the log-likelihood yields

$$ \begin{align}
\log p(Y = y \mid X_1= x_1, \ldots, X_p = x_p) & = \log\left(p(Y = y) \prod_{i=1}^p p(X_i = x_i \mid Y = y) \right) \\
& = \log p(Y = y) + \sum_{i=1}^p \log p(X_i = x_i \mid Y = y).
\end{align}$$

Algorithmically, Naive Bayes is easy to implement because we can use simple counting to estimate $p(Y = y)$ and $p(X_i = x_i \mid Y = y)$ as $X_i$ and $Y$ are both categorical variables.  For example,

$$p(Y = y) = \frac{\mbox{number of samples where label is }y}{\mbox{number of samples}}$$

and

$$p(X_i = x_i \mid Y = y) = \frac{\mbox{number of samples where feature }X_i\mbox{ is }x_i \mbox{ and }Y\mbox{ is } y}{\mbox{number of samples where label is }y}$$

Naive Bayes is naturally and easily parallelizable since the feature counting can be distributed across different processes.  

In spite of the simplified assumptions about the independence of the features, Naive Bayes classifiers have worked well in real-world applications such as spam filtering and document classification.  

**Questions**:

1. Naive Bayes intentionally makes an unrealistic assumption about $X_1, \ldots, X_p$ being independent to generate a linear model.  What is a similar model which doesn't rely on this assumption?  Why do we not use it instead of Naive Bayes?
1. What is the value of $p(X_i | Y)$ if there are no examples in this class?  How might you fix this?
1. Naive Bayes intentionally makes an unrealistic assumption about $X_1, \ldots, X_p$ being independent.  We could easily capture the correlation structure just by computing the full "tensor" of possibilities.  For example, for $p=2$, we would fill in the counts for each class of $Y$ in the matrix:

  <table>
    <col width="80">
    <col width="80">
    <col width="80">
  <tr>
  <td> </td>
  <td>$X_1=A$</td>
  <td>$X_1=B$</td>
  </tr>

  <tr>
  <td>$X_2=C$</td>
  <td></td>
  <td></td>
  </tr>

  <tr>
  <td>$X_2=D$</td>
  <td></td>
  <td></td>
  </tr>
  </table>

  Why might you not want to do this in general?

In Scikit-Learn, information about Naive Bayes 
can be found [here](http://scikit-learn.org/stable/modules/naive_bayes.html).  The various types of algorithms can all be found in the `sklearn.naive_bayes` module:
1. `GaussianNB` (for features that are assumed to have Gaussian likelihoods, not described here).
1. `MultinomialNB` (for discrete features).
1. `BernoulliNB` (for discrete features that are always binary/Boolean).
1. `ComplementNB` (described as being "particularly suited for imbalanced data sets").  

Note:  The Scikit-Learn implementations of these algorithms include a `partial_fit` method that allows for online learning or batch processing of large quantities of data.  

## Classifying mushrooms (an example)

Let's consider a data set that's available on the University of California Irvine __[Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)__.  This data set has 8124 instances of "hypothetical samples" corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota family.  There are 22 features in the data set, all of which are categorical.  Each mushroom species is identified as definitely edible, definitely poisonous, or of unknown edibility and therefore "not recommended".  For this data set, the latter class was combined with the poisonous one.  

Our goal here is to classify mushrooms as "edible" or "poisonous" based on features such as "cap-shape", "odor", and "habitat".  You can read more about this data set and feature information at the __[UCI Repository webpage](https://archive.ics.uci.edu/ml/datasets/Mushroom)__.  Otherwise, we won't go into the details about what all the features mean, except to remind you that they are all categorical features.  We will note that feature 11 ("stalk-root") has some "missing" data that has been encoded as a question mark "?", but we will just interpret that as another possible value for feature 11 in this case.  

We can download this data using the `pandas.read_csv` function.  

In [ ]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data',
                 header=None,
                 names=['edible', 'cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 
                        'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 
                        'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 
                        'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 
                        'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 
                        'population', 'habitat']   #  Column names taken from UCI description
                )

df.head()

The first column ("edible" vs "poisonous") is what we want to predict, so let's save that in a different variable, and then delete it from the DataFrame.  We'll translate this data from a string into a numeric field.  

In [ ]:
#  Define our 'target', i.e. what we want to predict in terms of numeric values, 1 if edible, 0 if not
target = df['edible'].apply(lambda x: 1 if x =='e' else 0).values

#  So then we can drop this "edible" column from the DataFrame
df.drop('edible', axis=1, inplace=True)

Before proceeding further, let's do a train/test split so that we can test our model later, holding back 20% of the data.  

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, 
                                                    random_state=42)

As for any machine learning algorithm, the categorical features need to be translated into numeric data.  In fact, so that the counts can be computed properly for all of the different feature values in the Naive Bayes model, we need to do a one-hot encoding for each feature.  Unfortunately, the `OneHotEncoder` transformer in the version of Scikit-Learn we use (version 0.19.x) doesn't allow for a _direct_ translation of text data into the one-hot encoding.  So we have to do this indirectly.  

[`LabelEncoder`](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html) will translate a single column of strings into numeric data.  For a feature with $n$ different values, the outputs of `LabelEncoder` will be integers in the range $0,\ldots, (n-1)$.  We want to change each column of our input into numeric data, so let's build a custom transformer that will apply (separate versions of) `LabelEnoder` to a set of columns.

In [ ]:
from sklearn.preprocessing import LabelEncoder

from sklearn.base import BaseEstimator, TransformerMixin

class MultiColumnEncoder(BaseEstimator, TransformerMixin):
    '''Applies multiple LabelEncoders to a DataFrame.  (Assumes input data is in the form of a 
    pandas DataFrame.)  
    
    Supply the column names when the transformer is created.  The input data is copied before 
    the transformations are performed on it.'''
    
    def __init__(self, trans_columns=None):
        self.trans_columns = trans_columns
        
    def fit(self, X, y=None):   #  Fit a LabelEncoder for each applicable column, store them for later use.
        self.encoders = {}
        for c in self.trans_columns:
            le = LabelEncoder()
            le.fit(X[c])
            self.encoders[c] = le
        return self
    
    def transform(self, X):     #  Use the fitted LabelEncoders to perform the transformations.
        X_copy = X.copy()       #  Copy the DataFrame first, and apply transformers to the copy.  
        for (c, le) in self.encoders.items():
            X_copy[c] = le.transform(X_copy[c])            
        return X_copy

A quick check to see that this works...

In [ ]:
columns_to_encode = X_train.columns

mce = MultiColumnEncoder(columns_to_encode)
mce.fit_transform(X_train).head()

We can then use `OneHotEncoder` on this transformed output, which will be the input to the Naive Bayes estimator.  So let's build a `Pipeline` that will perform the two transformations and fit the estimator.  Since the output of `OneHotEncoder` is a 0/1 matrix, in this case we can use the `BernoulliNB` estimator.  

(Note:  In more recent versions of Scikit-Learn, `OneHotEncoder` can _directly_ transform strings to one-hot encoded labels.)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import BernoulliNB

from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('mce', MultiColumnEncoder(columns_to_encode)),
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('bnb', BernoulliNB())
])

pipe.fit(X_train, y_train)

And how do we perform on the test data?

In [ ]:
pipe.score(X_test, y_test)

While 93.6% accuracy is quite good, this is a situation where accuracy isn't necessarily the "best" performance measure of a classifier (recall that the `score` method for a classifier defaults to "accuracy").  

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, classification_report

y_pred = pipe.predict(X_test)

precision_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

Nearly 10% of the samples (88 out of 915) identified as "edible" aren't...  It should go without saying that you don't want to use this classifier (or, to be honest, any other machine learning classifier) as a guide to what mushrooms you can safely eat.  

In case you didn't know/remember, we can get some additional metrics about our classifier's performance from Scikit-Learn's `classification_report`.  

In [ ]:
print(classification_report(y_test, y_pred, target_names=['poisonous', 'edible']))

**Questions** 

4. What happens if you try the `MultinomialNB` classifier instead of `BernoulliNB` for this data set?  
1.  Multinomial (or Bernoulli) Naive Bayes uses a smoothing parameter `alpha` in computing the parameters in the model.  See [here](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) for some more details.  What's the effect of altering `alpha` in `BernoulliNB` or `MultinomialNB` for this particular data set? And why? 
1.  How does the performance of `BernoulliNB` (or `MultinomialNB`) compare to a Logistic Regression classifier on this data?   

### Exit Tickets

1. Describe how you would implement Naive Bayes using MapReduce or Spark/PySpark for those familiar with these tools.
1. What hyperparameters in Naive Bayes might you need to tune using cross-validation?

### Spoilers

### Answers


1. Linear Regression requires loading all the data into memory.  Naive Bayes can work on streaming data and hence can be used in parallelized algorithms like MapReduce.
1. Without data, the conditional probability is either not defined (if no instances of $Y=y$ occur) or zero (if no instances of the pair $(X_i=x_i, Y=y)$ occur).  The way you get around this is called Bayesian smoothing.  For example,

   $$p(Y = y) = \frac{\mbox{number of samples where label is }y + p_0 n_0}{\mbox{number of samples} + n_0}$$

   where we assume some "prior" of rate $p_0$ with $n_0$ observed examples.

1. If each of $X_1,\ldots,X_n$ can take on $K$ possible values, then Naive Bayes has $Kp$ "bins" to fill.  The full "tensor" would have $K^p$ possibilities and most of those cells would be empty since it is likely that $n < K^p$.  This exponential growth in the number of "bins" is called the curse of dimensionality.  On the other hand, if $n \gg K^p$ you might want to do the full tensor.

4.  `MultinomialNB` gives slightly better accuracy (and precision) for this data set.  The decision functions of the two classifiers differ.  According to Scikit-Learn documentation, `BernoulliNB` "penalizes the non-occurrence of a feature $i$ that is an indicator for \[a class\], where the multinomial variant would simply ignore a non-occurring feature."  It's not always possible to use `BernoulliNB` (as your features must be binary), but when you can it's useful to compare `MultinomialNB` and `BernoulliNB` using whatever metric you are interested in optimizing.  
1.  In the Scikit-Learn implementations of (Multinomial and Bernoulli) Naive Bayes, the smoothing parameter `alpha` pushes all estimated probabilities (slightly) towards a uniform distribution, i.e. towards $1/p$ where $p$ is the number of features in the data set.  This prevents zero probabilities in further computations.  For _this_ mushroom data set, taking `alpha` closer to 0 improves the performance of the classifier.  In general, finding the right value of `alpha` for a data set can be done using cross-validation.  
1.  Logistic Regression will give 100% accuracy for this data set (try it out!).  With 22 features and enough samples in this case, LR is able to find a perfect separation between the two classes (edible and poisonous), even having not seen the testing data set during the fit process.  (I still don't think I'm going to trust it to tell me what I can eat or not!) 

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*